In [1]:
%load_ext autoreload
%autoreload 2

## initialization

In [2]:
from beam_search import create_dataset
from data import encode_line, load_vocabulary
from model import Transformer
import torch
import torch.nn as nn
import argparse

args = argparse.Namespace()
args.src_vocab = '../corpus/vocab_en_fr.txt'
args.tgt_vocab = '../corpus/vocab_en_fr.txt'
args.ckpt = './averaged_checkpoint.pt'
args.device = 'cpu'

batch_size = 16
beam_size = 5
length_penalty = 1
max_length = 256

source_vocabulary, _ = load_vocabulary(args.src_vocab)
target_vocabulary, target_vocabulary_rev = load_vocabulary(args.tgt_vocab)

bos = target_vocabulary["<s>"]
eos = target_vocabulary["</s>"]
model = Transformer(
    len(source_vocabulary),
    len(target_vocabulary),
    share_embeddings=True,
)
checkpoint = torch.load(args.ckpt)
model.load_state_dict(checkpoint["model"])
model.to(args.device)
model.eval()
n = 0
for param in model.parameters():
    # print(param.shape)
    n += param.numel()
print(n)

source_path = '../corpus/rep_test.en.tok'
dataset = create_dataset(
    source_path, source_vocabulary, batch_size, args.device)
ref_path = '../corpus/rep_test.fr'
with open(ref_path, 'r', encoding='utf-8') as file:
    ref = file.readlines()
ref[0]

209129472


'Cette fois-ci, la baisse est due à la chute des actions au Wall Street.\n'

## helper functions

In [33]:
def view(hypotheses):
    for hypo in hypotheses:
        score = hypo[0].item()
        tokens = hypo[1]
        if tokens and tokens[-1] == eos:
            tokens.pop(-1)
        tokens = [target_vocabulary_rev[token_id] for token_id in tokens]
        print(f"{score:.4f}", " ".join(tokens))

## inference analysis

In [3]:
from my_beam_search import beam_search

In [34]:
batch = next(iter(dataset))

In [35]:
batch.shape

torch.Size([16, 52])

In [4]:
from utils import *

### original version

In [ ]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    result = beam_search(
        model, batch[0:2], bos, eos, rep_penalty=False)

In [36]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    result = beam_search(
        model, batch, bos, eos, rep_penalty=True, naive_penalty=2, penalty_mask=penalty_mask)

In [38]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    result2 = beam_search(
        model, batch, bos, eos, rep_penalty=False)

result is a batch of data, whose element is (score, sentence)

In [39]:
view(result2[10])

-0.2172 Mais le Congrès peut bloquer la libération du montant restant ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard pour les 3￭ 5￭ 0 milliards de dollars finaux ￭, s ￭'￭ il a l ￭'￭ impression que le programme ne remplit pas sa fonction ￭.
-0.2183 Mais le Congrès peut bloquer la libération de la somme rest￭ ante ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard pour les 3￭ 5￭ 0 milliards de dollars finaux ￭, s ￭'￭ il a l ￭'￭ impression que le programme ne remplit pas sa fonction ￭.
-0.2217 Mais le Congrès peut bloquer la libération du montant restant ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard pour les 3￭ 5￭ 0 milliards de dollars finaux ￭, s ￭'￭ il a l ￭'￭ impression que le programme n ￭'￭ remplit pas sa fonction ￭.
-0.2230 Mais le Congrès peut bloquer la libération de la somme rest￭ ante ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard ￭, la somme finale de 3￭

In [37]:
view(result[10])

-0.2924 Mais le Congrès peut bloquer la libération de la somme rest￭ ante ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard pour les 3￭ 5￭ 0 milliards de dollars finaux ￭, s ￭'￭ il a l ￭'￭ impression que le programme ne remplit pas sa fonction ￭.
-0.2941 Mais le Congrès peut bloquer la libération du montant restant ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard pour les 3￭ 5￭ 0 milliards de dollars finaux ￭, s ￭'￭ il a l ￭'￭ impression que le programme ne remplit pas sa fonction ￭.
-0.2972 Mais le Congrès peut bloquer la libération du montant restant ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard pour les 3￭ 5￭ 0 milliards de dollars finaux ￭, s ￭'￭ il a l ￭'￭ impression que le programme n ￭'￭ remplit pas sa fonction ￭.
-0.3086 Mais le Congrès peut bloquer la libération du montant restant ￭, pour une série de 1￭ 0￭ 0 milliards de dollars supplémentaires et plus tard ￭, les 3￭ 5￭ 0 milliards de 

Maybe what we want is 

`Cette fois ￭, la chute des actions de Wall Street est responsable de la baisse ￭.`

However, it is not the second candidate

### naive version

In [12]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    penalized_result = beam_search(
        model, batch[0:2], bos, eos, rep_penalty=True, naive_penalty=5)

In [13]:
view(penalized_result[0])

-0.3882 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est responsable ￭.
-0.4981 Cette fois ￭-￭ ci ￭, la baisse des actions de Wall Street en est responsable ￭.
-0.5306 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est la cause ￭.
-0.7309 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ origine de cette baisse ￭.
-0.7371 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭'￭ origine de cette baisse ￭.
-0.7391 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ œuvre ￭.
-0.8594 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ origine de la baisse ￭.
-0.9045 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭’￭ origine de cette diminution ￭.
-0.9075 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est à l ￭'￭ origine de cette diminution ￭.


### mask version

In [3]:
from my_beam_search import calc_mask


penalty_mask, leading_marker = calc_mask(target_vocabulary)

In [6]:
idx = target_vocabulary['pourcent']
penalty_mask[idx]

KeyError: 'pourcent'

In [27]:
mask_idx = penalty_mask.nonzero().squeeze().tolist()

In [31]:
escape_terms = [tgt_vocab_rev[i] for i in range(32000) if i not in mask_idx]

In [32]:
'du' in escape_terms

True

In [64]:
leading_marker

tensor([0., 0., 0.,  ..., 1., 0., 0.])

In [15]:
torch.save(penalty_mask, 'penalty_mask.pt')

In [1]:
penalty_mask = torch.load('penalty_mask.pt')

In [17]:
# we just take small batch_size for test
# _ = beam_search(model, batch[None, 0], bos, eos)
with torch.no_grad():
    penalized_result = beam_search(
        model, batch[0:2], bos, eos, rep_penalty=True, naive_penalty=2, penalty_mask=penalty_mask)

In [18]:
view(penalized_result[0])

-0.2389 Cette fois ￭, la chute des actions de Wall Street en est la cause ￭.
-0.2528 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street en est la cause ￭.
-0.2846 Cette fois ￭, la chute des actions de Wall Street est responsable de la baisse ￭.
-0.3051 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est responsable de la baisse ￭.
-0.3103 Cette fois ￭, la baisse des actions de Wall Street est responsable de la chute ￭.
-0.3454 Cette fois ￭-￭ ci ￭, la chute des actions de Wall Street est responsable de la chute ￭.
-0.3845 Cette fois ￭, la chute des actions de Wall Street est responsable de la baisse des cours ￭.


## evaluation

In [6]:
def experi(model, dataset, **kwarg):
    isMask = 'penalty_mask' in kwarg
    filename = '_'.join(
        [f'{key}_{value}' for key, value in kwarg.items() if key != 'penalty_mask'])+('_mask' if isMask else '')+'.fr.tok'
    print(filename)
    with open('./corpus/'+filename, 'w', encoding='utf-8') as file:
        with torch.no_grad():
            for batch in dataset:
                result = beam_search(model, batch, bos, eos, **kwarg)
                # result = (batch_size, ), hypo = (score, sent)
                for hypotheses in result:
                    tokens = hypotheses[0][1]
                    if tokens and tokens[-1] == eos:
                        tokens.pop(-1)
                    tokens = [target_vocabulary_rev[token_id]
                              for token_id in tokens]
                    print(" ".join(tokens), file=file, flush=True)

In [7]:
kwarg = {
    'naive_penalty': 3
}
experi(model, dataset, **kwarg)

naive_penalty_3.fr.tok


In [10]:
kwarg = {
    'naive_penalty': 3,
    'penalty_mask': penalty_mask,
}
experi(model, dataset, **kwarg)

naive_penalty_3_mask.fr.tok


In [11]:
kwarg = {
    'naive_penalty': 3,
    'penalty_mask': penalty_mask,
    'penalty_decay': 0.9
}
experi(model, dataset, **kwarg)

naive_penalty_3_penalty_decay_0.9_mask.fr.tok


In [13]:
kwarg = {
    'naive_penalty': 2
}
experi(model, dataset, **kwarg)

naive_penalty_2.fr.tok


In [15]:
kwarg = {
    'naive_penalty': 2,
    'penalty_mask': penalty_mask,
}
experi(model, dataset, **kwarg)

naive_penalty_2_mask.fr.tok


In [14]:
kwarg = {
    'naive_penalty': 2,
    'penalty_mask': penalty_mask,
    'penalty_decay': 0.9
}
experi(model, dataset, **kwarg)

naive_penalty_2_penalty_decay_0.9_mask.fr.tok


In [16]:
kwarg = {
    'rep_penalty': False
}
experi(model, dataset, **kwarg)

rep_penalty_False.fr.tok


In [44]:
kwarg = {
    'naive_penalty': 1000000,
    'penalty_mask': penalty_mask,
}
experi(model, dataset, **kwarg)

naive_penalty_1000000_mask.fr.tok


In [7]:
import os
import subprocess
import json
from wordrep.utils import BaseRepDetector


def eval_score(directory='./corpus/', ref_path='../corpus/rep_test.fr'):
    c = "sacrebleu"  # Replace with your desired Bash command
    rep_detector = BaseRepDetector('fr')
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            # Process the file
            if filename.endswith('fr'):
                path = os.path.join(directory, filename)
                command = f'{c} {path} -i {ref_path}'
                # Execute the command
                process = subprocess.Popen(
                    command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                # Wait for the command to finish and capture the output
                output, _ = process.communicate()
                # Decode the output
                output = output.decode('utf-8')
                o = json.loads(output)
                reps = rep_detector.detect_corpus(path, vis=False)
                print(filename, f"bleu={o['score']}, rep={len(reps)}")

In [8]:
eval_score()

naive_penalty_2_mask.fr bleu=26.2, rep=444
naive_penalty_10_mask.fr bleu=24.6, rep=67
naive_penalty_100_mask.fr bleu=24.6, rep=66
naive_penalty_3_mask.fr bleu=25.8, rep=381
naive_penalty_3_penalty_decay_0.9_mask.fr bleu=26.4, rep=522
naive_penalty_2_penalty_decay_0.9_mask.fr bleu=26.4, rep=541
naive_penalty_2.fr bleu=23.5, rep=390
naive_penalty_3.fr bleu=21.7, rep=305
